# Explore Xeno-Canto M Dataset 


In [1]:
# switch to parent directory
import os
os.chdir("../..")

In [2]:
# get current working directory
cwd = os.getcwd()
print(cwd)

/workspace


# Load database with ESC50 datamodule

## Spectrogram

In [3]:
from birdset.datamodule.components.event_decoding import EventDecoding
from birdset.datamodule.components.transforms import PreprocessingConfig, BirdSetTransformsWrapper
from birdset.datamodule.base_datamodule import DatasetConfig
from birdset.datamodule.esc50_datamodule import ESC50DataModule

transforms = BirdSetTransformsWrapper(model_type='vision', decoding=EventDecoding(sampling_rate=32000), preprocessing=PreprocessingConfig(n_fft=1024, hop_length=79, n_mels=128))
dataset_config = DatasetConfig(
    data_dir="/workspace/data",
    dataset_name="esc50",
    hf_path="ashraq/esc50",
    hf_name="default",
    seed=42,
    n_classes=50,
    n_workers=1,
    val_split=0.2,
    task="multiclass",
    subset=None,
    sampling_rate=32000,
)

dm = ESC50DataModule(dataset=dataset_config, transforms=transforms)


In [4]:
dm.prepare_data()
print(dm.len_trainset)
print(dm.num_classes)

Saving the dataset (0/2 shards):   0%|          | 0/1280 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

1280
50


In [5]:
dm.setup(stage='fit')
train_dataloader = dm.train_dataloader

In [6]:
from matplotlib import pyplot as plt

# get first batch
batch = next(iter(train_dataloader()))
print(batch.keys())
print(batch['input_values'].shape)
spectorgram = batch['input_values'][0]
plt.imshow(spectorgram.squeeze().numpy())

dict_keys(['input_values', 'labels'])
torch.Size([32, 1, 128, 1024])


# Learning about spectorgram conversion

## Load dataset as waveform

In [26]:
from birdset.datamodule.components.event_decoding import EventDecoding
from birdset.datamodule.components.transforms import PreprocessingConfig, BirdSetTransformsWrapper
from birdset.datamodule.base_datamodule import DatasetConfig
from birdset.datamodule.birdset_datamodule import BirdSetDataModule



transforms = BirdSetTransformsWrapper(decoding=EventDecoding(sampling_rate=32000))
dataset_config = DatasetConfig(
    data_dir="/workspace/data",
    dataset_name="xenocanto_m_withoutnd",
    hf_path="DBD-research-group/birdset_v1",
    hf_name="xenocanto_M_withoutnd",
    seed=42,
    n_classes=410,
    n_workers=1,
    val_split=0.05,
    task="multiclass",
    subset=None,
    sampling_rate=32000,
)

dm_waveform = BirdSetDataModule(dataset=dataset_config, transforms=transforms)
dm_waveform.prepare_data()
print(dm_waveform.len_trainset)
print(dm_waveform.num_classes)
dm_waveform.setup(stage='fit')

Saving the dataset (0/1 shards):   0%|          | 0/305674 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/16160 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4001 [00:00<?, ? examples/s]

305674
410


In [55]:
# get one sample from the dataset
sample = dm_waveform.train_dataset[8]
print(sample['input_values'].shape)
# print the converted label
print('Label: ',dm_waveform.train_dataset.features['labels'].names[sample['labels']])
sample

torch.Size([1, 160000])
Label:  apapan


{'input_values': tensor([[-0.0027, -0.0041, -0.0060,  ...,  0.0000,  0.0000,  0.0000]]),
 'labels': 358}

In [56]:
# listen to sample
import IPython.display as ipd
waveform = sample['input_values'].squeeze().numpy()
ipd.Audio(waveform, rate=32000)


## Convert waveform to spectrogram

In [57]:
import librosa
def plot_waveform(waveform, sr, title="Waveform", ax=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    if ax is None:
        _, ax = plt.subplots(num_channels, 1)
    ax.plot(time_axis, waveform[0], linewidth=1)
    ax.grid(True)
    ax.set_xlim([0, time_axis[-1]])
    ax.set_title(title)


def plot_spectrogram(specgram, title=None, ylabel="freq_bin", ax=None):
    if ax is None:
        _, ax = plt.subplots(1, 1)
    if title is not None:
        ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto", interpolation="nearest")

<class 'numpy.ndarray'>


In [65]:
import torchaudio

waveform_tensor = sample['input_values']

# Define transform
spectrogram_transforms = torchaudio.transforms.Spectrogram(n_fft=512)

# Perform transform
spec = spectrogram_transforms(waveform_tensor)

In [68]:
import torch
fig, axs = plt.subplots(2, 1)
plot_waveform(waveform_tensor, 32000, title="Original waveform", ax=axs[0])
plot_spectrogram(spec[0], title="spectrogram", ax=axs[1])
fig.tight_layout()